In [417]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Input, Dropout

In [418]:
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\titanic.csv')

In [419]:
# Sets null values equal to the median of that column
def handle_null_median(df):
    # Need to set inplace=True, so it doesn't create a copy of the dataframe. Tried without and this led to null-values not being removed
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)

    df['Embarked'].fillna('S', inplace=True)

    return  df

df = handle_null_median(df)
df['Sex'] = df['Sex'].replace(['female', 'male'], [0,1])
df['FamilyMembersCount'] = df['Parch'] + df['SibSp'] + 1
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

In [420]:
def bins(df): # This is kind of feature engineering as well
    df['AgeGroup'] = 0
    df.loc[df['Age'] < 16.336, 'AgeGroup' ] = 0
    df.loc[(df['Age'] >= 16.336) & (df['Age'] < 32.252), 'AgeGroup'] = 1
    df.loc[(df['Age'] >= 32.252) & (df['Age'] < 48.168), 'AgeGroup'] = 2
    df.loc[(df['Age'] >= 48.168) & (df['Age'] < 64.084), 'AgeGroup'] = 3
    df.loc[df['Age'] >= 64.084, 'AgeGroup'] = 4

    df = df.drop('Age', axis=1)

    # Could also create bins for fare, but not sure Fare is needed.
    return df

df = bins(df)

df['Pclass'] = df['Pclass'].astype('category')
df['Sex'] = df['Sex'].astype('category')
df['Embarked'] = df['Embarked'].astype('category')
df['AgeGroup'] = df['AgeGroup'].astype('category')

df.head()

,Survived,Pclass,Sex,Fare,Embarked,FamilyMembersCount,AgeGroup
0,0,3,1,7.2500,S,2,1
1,1,1,0,71.2833,C,2,2
2,1,3,0,7.9250,S,1,1
3,1,1,0,53.1000,S,2,2
4,0,3,1,8.0500,S,1,2


In [421]:
features = df.drop('Survived',axis=1)
targets = df['Survived']

targets_onehot = pd.get_dummies(targets)
features_onehot = pd.get_dummies(features)
targets_onehot = targets_onehot.astype('float32')
features_onehot = features_onehot.astype('float32')

features_onehot.head()

,Fare,FamilyMembersCount,Pclass_1,Pclass_2,Pclass_3,Sex_0,Sex_1,Embarked_C,Embarked_Q,Embarked_S,AgeGroup_0,AgeGroup_1,AgeGroup_2,AgeGroup_3,AgeGroup_4
0,7.250000,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,71.283302,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,7.925000,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,53.099998,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,8.050000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [422]:
features_onehot = normalize(features_onehot)

train_features, test_features, train_targets, test_targets = train_test_split(features_onehot, targets_onehot, test_size=0.25, random_state=42)

In [423]:
train_features_tensor = tf.convert_to_tensor(train_features)
test_features_tensor = tf.convert_to_tensor(test_features)
train_targets_tensor = tf.convert_to_tensor(train_targets)
test_targets_tensor = tf.convert_to_tensor(test_targets)

## I've trained this model on the same dataset as the classification task in the 2nd mandatory assignment so I can compare result. On average the 2nd Nerual Network moddel outperforms the other one.

# NN Architecture 1

In [434]:
model1 = Sequential()

model1.add(Input(15))

model1.add(Dense(30, activation='relu'))
model1.add(Dense(30, activation='relu'))
model1.add(Dense(15, activation='relu'))

model1.add(Dense(2, activation='sigmoid'))

In [435]:
opt1 = Adam(learning_rate=0.01)
model1.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

In [436]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=200, mode='auto')
model1.fit(train_features_tensor, train_targets_tensor, epochs=250, batch_size=10, callbacks=[callback])

Epoch 1/250
67/67 [==============================] - 0s 2ms/step - loss: 0.6439 - accuracy: 0.6048
Epoch 2/250
67/67 [==============================] - 0s 2ms/step - loss: 0.5571 - accuracy: 0.7156
Epoch 3/250
67/67 [==============================] - 0s 2ms/step - loss: 0.5129 - accuracy: 0.7605
Epoch 4/250
67/67 [==============================] - 0s 2ms/step - loss: 0.4975 - accuracy: 0.7799
Epoch 5/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4876 - accuracy: 0.7859
Epoch 6/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4685 - accuracy: 0.7889
Epoch 7/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4528 - accuracy: 0.8039
Epoch 8/250
67/67 [==============================] - 0s 2ms/step - loss: 0.4615 - accuracy: 0.8129
Epoch 9/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4959 - accuracy: 0.7650
Epoch 10/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4580 - accuracy: 0.8024
Epoch 11/

In [437]:
model1.evaluate(train_features_tensor, train_targets_tensor)

21/21 [==============================] - 0s 1ms/step - loss: 0.3630 - accuracy: 0.8249


[0.36296966671943665, 0.82485032081604]

In [438]:
model1.evaluate(test_features_tensor, test_targets_tensor)

7/7 [==============================] - 0s 2ms/step - loss: 0.7972 - accuracy: 0.8117


[0.7972198724746704, 0.8116592168807983]

This architecture performs slightly worse than the on in the 2nd assignment

# NN Architecture 2

In [591]:
model2 = Sequential()

model2.add(Input(15))

model2.add(Dense(30, activation='relu'))

model2.add(Dense(15, activation='relu'))
model2.add(Dense(2, activation='sigmoid'))

In [592]:
opt1 = Adam()
model2.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

In [593]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=200, mode='auto')
model2.fit(train_features_tensor, train_targets_tensor, epochs=250, batch_size=15, callbacks=[callback])

Epoch 1/250
45/45 [==============================] - 0s 1ms/step - loss: 0.6892 - accuracy: 0.5734
Epoch 2/250
45/45 [==============================] - 0s 2ms/step - loss: 0.6765 - accuracy: 0.6213
Epoch 3/250
45/45 [==============================] - 0s 1ms/step - loss: 0.6627 - accuracy: 0.6213
Epoch 4/250
45/45 [==============================] - 0s 1ms/step - loss: 0.6453 - accuracy: 0.6213
Epoch 5/250
45/45 [==============================] - 0s 1ms/step - loss: 0.6272 - accuracy: 0.6213
Epoch 6/250
45/45 [==============================] - 0s 1ms/step - loss: 0.6041 - accuracy: 0.6572
Epoch 7/250
45/45 [==============================] - 0s 999us/step - loss: 0.5772 - accuracy: 0.6916
Epoch 8/250
45/45 [==============================] - 0s 724us/step - loss: 0.5568 - accuracy: 0.7665
Epoch 9/250
45/45 [==============================] - 0s 692us/step - loss: 0.5441 - accuracy: 0.7695
Epoch 10/250
45/45 [==============================] - 0s 743us/step - loss: 0.5350 - accuracy: 0.7740
E

In [594]:
model2.evaluate(train_features_tensor, train_targets_tensor)

21/21 [==============================] - 0s 576us/step - loss: 0.3933 - accuracy: 0.8308


[0.3933236002922058, 0.8308383226394653]

In [595]:
model2.evaluate(test_features_tensor, test_targets_tensor)

7/7 [==============================] - 0s 667us/step - loss: 0.4749 - accuracy: 0.8161


[0.4749326705932617, 0.8161435127258301]

This model has less layers than the previous model, but performs slightly better than the previous one. This model performs worse than the one in the 2nd assignment